Now, let's finally look at that divorced dataset.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
df = pd.read_csv('../data/divorces_2000-2015_translated.csv')
print(df.shape)
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('.', '')
df.head()

(4923, 41)


,divorce_date,type_of_divorce,nationality_partner_man,dob_partner_man,place_of_birth_partner_man,birth_municipality_of_partner_man,birth_federal_partner_man,birth_country_partner_man,age_partner_man,residence_municipality_partner_man,...,marriage_certificate_municipality,marriage_certificate_federal,level_of_education_partner_man,employment_status_partner_man,level_of_education_partner_woman,employment_status_partner_woman,marriage_duration,marriage_duration_months,num_children,custody
0,9/6/06,Necesario,MEXICANA,18/12/75,XALAPA - ENRIQUEZ,XALAPA,VERACRUZ,MEXICO,30.0,XALAPA,...,XALAPA,VERACRUZ,SECUNDARIA,OBRERO,SECUNDARIA,EMPLEADO,5.0,NaN,1.0,NaN
1,1/2/00,Voluntario,MEXICANA,NaN,NaN,NaN,NaN,NaN,47.0,NaN,...,XALAPA,VERACRUZ,PREPARATORIA,ESTABLECIMIENTO,PREPARATORIA,EMPLEADO,NaN,NaN,NaN,NaN
2,1/2/05,Necesario,MEXICANA,22/2/55,XALAPA - ENRIQUEZ,XALAPA,VERACRUZ,MEXICO,49.0,NaN,...,XALAPA,VERACRUZ,PREPARATORIA,OBRERO,NaN,TRABAJADOR POR CUENTA PROPIA EN VIA PUBLICA,NaN,NaN,NaN,NaN
3,1/2/06,Necesario,MEXICANA,20/1/64,XALAPA - ENRIQUEZ,XALAPA,VERACRUZ,MEXICO,42.0,XALAPA,...,XALAPA,VERACRUZ,PROFESIONAL,EMPLEADO,PREPARATORIA,EMPLEADO,18.0,NaN,2.0,MADRE
4,1/2/06,Necesario,MEXICANA,30/10/75,XALAPA - ENRIQUEZ,XALAPA,VERACRUZ,MEXICO,30.0,COATEPEC,...,XALAPA,VERACRUZ,PROFESIONAL,EMPLEADO,PREPARATORIA,NO TRABAJA,7.0,NaN,2.0,MADRE


Yeah, once again we need to do a lot of sprucing. We'll definitely err on the side of making things as simple as possible.

In [5]:
 i = 0

In [9]:
df[df.columns[i]].value_counts() #Yeah, we don't care about this... way too detailed for our purposes. Do note though
#the range of them

divorce_date
9/10/09     9
5/9/05      8
25/2/09     8
21/11/03    8
3/11/04     8
           ..
8/10/13     1
8/10/14     1
8/11/01     1
8/11/06     1
31/12/13    1
Name: count, Length: 2596, dtype: int64

In [10]:
min(df['divorce_date']), max(df['divorce_date'])

('1/10/01', '9/9/14')

In [ ]:
df.drop(df.column[])